#**Project No:** *1*</font><br>

#**Title:** <font color='red'>*PageRank using the Power Method*</font><br /><br />

* [Data Collection & Preparation](#dcp)
* [PageRank func using the Power Method](#prpm)

[PART 1](#part1):
  * [Q.1.a.](#q1a)
  * [Q.1.b.](#q1b)
  * [Q.1.c.](#q1c)

[PART 2](#part2):
  * [Q.2.a.](#q2a)
  * [Q.2.b.](#q2b)
  * [Q.2.c.](#q2c)
  * [Q.2.d.](#q2d)
  * [Q.2.e.](#q2e)

  <br>

# References
* [Project Specs](https://github.com/geoav74/aueb_projects/blob/main/numerical_opt_%26_large-scale_linear_algebra/page-rank_power-method/eig_PR_eng.pdf)
* [Deeper Inside PageRank](https://github.com/geoav74/aueb_projects/blob/main/numerical_opt_%26_large-scale_linear_algebra/page-rank_power-method/pagerank.pdf)


<a name="part1"></a>
# <font color=2B60DE>**PART 1**

<a name="dcp"></a>
>## <font color=2B60DE>**Data Collection & Preparation**

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import math
import time

In [ ]:
# Mounting my google drive to the Colab's environment so to get the data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Defining the file path from my google drive directory
path_data = '/content/drive/MyDrive/Colab Notebooks/numerical/stanweb.dat'

In [ ]:
# reading .dat file as a df
data = pd.read_csv(path_data, sep='\t' , header = None)

In [ ]:
# formating our data
data.columns = ['fromNode','toNode', 'Proba']
data[['fromNode','toNode']] = data[['fromNode','toNode']].astype(int)

In [ ]:
print(f'fromNode datatype: {data.dtypes[0]} --- toNode datatype: {data.dtypes[1]} --- proba datatype: {data.dtypes[2]}')
print(f'The shape of our dataframe: {data.shape}')
print()
# sample of our data
data.head()

fromNode datatype: int64 --- toNode datatype: int64 --- proba datatype: float64
The shape of our dataframe: (2382912, 3)



,fromNode,toNode,Proba
0,1,6548,0.500000
1,1,15409,0.500000
2,2,252915,0.032258
3,2,246897,0.032258
4,2,251658,0.032258


In [ ]:
#sort by fromNode and toNode
data_array = np.array(data.sort_values(by = ['fromNode','toNode']))
data_array.shape

(2382912, 3)

In [ ]:
# looking for dangling nodes
data_new = np.array(data.groupby(['fromNode'])['toNode'].count().reset_index('fromNode'))

print(f'Dangling Nodes = {np.max(data.fromNode) - len(data_new)}')

Dangling Nodes = 172


In [ ]:
DangingNodes = np.zeros((len(data_new), 2))
a_vector = np.zeros((np.max(data.fromNode), 1))

# find the indices where there is a missing node
for i in range(1, len(data_new)):
    DangingNodes[i-1][0] = i
    DangingNodes[i-1][1] = data_new[i][0] - data_new[i-1][0]

DangingNodes = DangingNodes.astype(int)

DangingNodes = DangingNodes[DangingNodes[:, 1] == 2]

In [ ]:
# fixing indexing
for i in range(len(DangingNodes)):
    a_vector[DangingNodes[i][0] + i] = 1

a_vector = a_vector.astype(int)

print(f'a_vector shape: {a_vector.shape}')
print(f'DangingNodes shape: {DangingNodes.shape}')

a_vector shape: (281903, 1)
DangingNodes shape: (172, 2)


In [ ]:
# sampling the dangling
DangingNodes[:5, 0]

array([ 279, 1968, 3186, 4446, 4490])

In [ ]:
#create the initialization vector
e = np.ones((np.max(data['fromNode']), 1))
E_array = (1 / np.max(data['fromNode'])) * e[:, 0]
n = E_array.shape[0]

index = np.zeros((n, 1))
for i in range(index.shape[0]):
    index[i,0]=i

E = csr_matrix( (E_array, (index[:,0], np.zeros(n))), shape = (n,1) )
E

<281903x1 sparse matrix of type '<class 'numpy.float64'>'
	with 281903 stored elements in Compressed Sparse Row format>

In [ ]:
print(f'e shape: {e.shape}')
print(f'E_array shape: {E_array.shape}')
print(f'index shape: {index.shape}')
print(f'E shape: {E.shape}')

e shape: (281903, 1)
E_array shape: (281903,)
index shape: (281903, 1)
E shape: (281903, 1)


In [ ]:
#create the transition matrix P
P = csr_matrix((data_array[:, 2], (data_array[:, 0] - 1, data_array[:, 1] - 1)), shape = (n,n))
P

<281903x281903 sparse matrix of type '<class 'numpy.float64'>'
	with 2312497 stored elements in Compressed Sparse Row format>

<a name="prpm"></a>
>## <font color=2B60DE>**PageRank func using the Power Method**

In [ ]:
def PR_PM(P, alpha, taph):
  '''
  USAGE: to compute the pageRank among connections using the power method

  INPUT:
        @P     : transition matrix
        @alpha : constant
        @taph  : stopping criterion
  OUTPUT:
        @pr_post      : pageRank
        @iterations   : iterations
        @total_time   : algorith's runtime
        @convergeFast : list containing the nodes that converge during the
                        first iteration
  '''

  n = P.shape[0]

  # a list for storing nodes that have converged in 1st iteration
  convergeFast = []

  # rough estimate of the number of iterations needed to converge to a tolerance level 'taph'
  k =  int((math.log10(taph) / math.log10(alpha)))

  # initializing the pageRank
  pr_pre = csr_matrix((np.ones((n, 1))[:,0], (index[:, 0], np.zeros(n))), shape = (n, 1)) / n
  E = csr_matrix((np.ones((n, 1))[:, 0], (index[:, 0], np.zeros(n))), shape = (n, 1)) / n

  iterations = 1

  start = time.time()

  #running the power method until tolerance
  while iterations <= k:

      pre_dot_a_vector = (alpha * pr_pre.T).dot(a_vector)

      # computation of the 2 parts of the equation
      part2 = (pre_dot_a_vector + (1 - alpha))[0,0] * E.T
      part1 = (alpha * pr_pre.T).dot(P)

      # pageRang(iteration + 1) computation
      pr_post = (part1 + part2).T

      if iterations < 2:
        for i in range(n):
          if np.linalg.norm((pr_pre[i] - pr_post[i].toarray())) < taph:
                    convergeFast.append(i)

      # halt condition when reaches the tolerance
      if np.linalg.norm((pr_pre - pr_post).toarray(), ord=1) <= taph:
          break

      # setting the last calculated pageRank as previous
      pr_pre = pr_post.copy()

      # increasing the counter of iterations
      iterations += 1

  stop = time.time()
  total_time = stop - start

  return (pr_post.toarray()[:, 0], iterations - 1, total_time, convergeFast)

<a name="q1a"></a>
>## <font color=2B60DE>**Question a (PageRank using a = 0.85)**

In [ ]:
alpha = 0.85
tolerance = 10**(-8)

pageRank, iterations, timer, convergeFast = PR_PM(P, alpha, tolerance)

In [ ]:
print(f'iterations to converge: {iterations}')
print(f'executing time: {timer:.4f} secs')

results = pd.DataFrame({'Nodes': (index[:, 0] + 1).astype(int), 'PageRank': pageRank})
results = results.sort_values(by = ['PageRank', 'Nodes'], ascending = False).set_index(index[:, 0].astype(int))

results[:10]

iterations to converge: 90
executing time: 100.4011 secs


,Nodes,PageRank
0,89073,0.011303
1,226411,0.009288
2,241454,0.008297
3,262860,0.003023
4,134832,0.003001
5,234704,0.002572
6,136821,0.002454
7,68889,0.002431
8,105607,0.002397
9,69358,0.002364


<a name="q1b"></a>
>## <font color=2B60DE>**Question b (PageRank using a = 0.99)**

In [ ]:
alpha = 0.99
tolerance = 10**(-8)

pageRank_b, iterations_b, timer_b, convergeFast_b = PR_PM(P, alpha, tolerance)

In [ ]:
print(f'iterations to converge: {iterations_b}')
print(f'executing time: {timer_b:.4f} secs')

results_b = pd.DataFrame({'Nodes': (index[:, 0] + 1).astype(int), 'PageRank': pageRank_b})
results_b = results_b.sort_values(by = ['PageRank', 'Nodes'], ascending = False).set_index(index[:, 0].astype(int))

results_b[:10]

iterations to converge: 1391
executing time: 386.8185 secs


,Nodes,PageRank
0,89073,0.009187
1,281772,0.009112
2,174665,0.007689
3,226411,0.004514
4,179645,0.004073
5,271409,0.003872
6,262860,0.003485
7,136821,0.002821
8,68889,0.002790
9,77988,0.002676


In [ ]:
# if the nodes are not the same in both situations, the ranking has changed and
# we get a false value
results.Nodes[:50] == results_b.Nodes[:50]

0      True
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33     True
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
Name: Nodes, dtype: bool

We observe that using a=0.99, **the ranking has changed**

<a name="q1c"></a>
>## <font color=2B60DE>**Question c**

In [ ]:
print(f'the number of nodes that have converged in the 1st iteration using a=0.85 is: {len(convergeFast)}')
print(f'the number of nodes that have converged in the 1st iteration using a=0.99 is: {len(convergeFast_b)}')

the number of nodes that have converged in the 1st iteration using a=0.85 is: 14534
the number of nodes that have converged in the 1st iteration using a=0.99 is: 14522


In [ ]:
print(f'The following nodes have converged in 1st iteration of the power method using a=0.85:')
for idx,node in enumerate(results.Nodes[:50]):
  if node in convergeFast:
    print(f'position {idx} -- node {node}')

The following nodes have converged in 1st iteration of the power method using a=0.85:
position 15 -- node 95163
position 25 -- node 235496
position 33 -- node 77999


In [ ]:
print(f'The following nodes have converged in 1st iteration of the power method using a=0.99:')
for idx,node in enumerate(results_b.Nodes[:50]):
  if node in convergeFast_b:
    print(f'position {idx} -- node {node}')

The following nodes have converged in 1st iteration of the power method using a=0.99:
position 13 -- node 95163
position 24 -- node 235496
position 33 -- node 77999
position 36 -- node 119822


The sence is that the highest ranking nodes need more iterations to converge than the lower ones because the lower rank nodes tend to have more outgoing than incoming connections.

<a name="part2"></a>
# <font color='red'>**PART 2**

<a name="q2a"></a>
>## <font color='red'>**Question 2.a.**

Considering $\color{red}n$ as the size of the nodes in our graph,  $\color{red}{n_j}$  the number of outbound connections of node  $\color{red}j$  and  $\color{red}{S_i}$ the set of all the nodes that have outbound connections to node  $\color{red}i$,  we can state that the stationary form of the $i^{th}$ node's PageRank is:
$$\pi_i = \frac{1-\alpha}{n}+\alpha\sum_{j \in S_i}\frac{\pi_j}{n_j} $$
</br>
Adding a node, the above function will take the form:
$$\tilde\pi_i = \frac{1-\alpha}{n+1}+\alpha\sum_{j \in S_i}\frac{\pi_j}{n_j} $$
</br>
And the page rank of the new node will take a final form:
$$\tilde\pi_i = \frac{1-\alpha}{n+1}$$
</br>
The PageRank of the older pages should not change significantly. This is because the new node does not affect the probability distribution, as it is not connected to any other pages and the number n is significantly big.

<a name="q2b"></a>
>## <font color='red'>**Question 2.b.**

By adding a new node Y that only connects to the X the page rank of X is expected to increase, while the page rank of the page Y is expected to be low. This is because the page rank algorithm considers both the number of incoming links and the importance of those links when computing the ranking of a page.


<a name="q2c"></a>
>## <font color='red'>**Question 2.c.**

I think that the best way to do increase X's page rank would be to connect Y and Z pages to the X taking into account that Y and Z have significant page ranks.

<a name="q2d"></a>
>## <font color='red'>**Question 2.d.**

Considering n a very large number, X's page rank depends on its inbound connections. Adding links from node X, Y, Z to older popular nodes will not change  X's page rank.

<a name="q2e"></a>
>## <font color='red'>**Question 2.e.**

Page X should have in-nodes from other pages with significant page ranks. A moddified link farm could provide exactly that e.g. as the cluster members lineary increase their page rank increases. While X having high valued nodes, its page ranking will be significant.